In [ ]:
# Prediction with OpenAI

In [ ]:
!pip install openai
!pip install langchain
!pip install eodhd
!pip install config

import re
import requests
import pandas as pd
import config as cfg
from eodhd import APIClient
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:

api_key = '<YOUR API KEY>'

In [ ]:

resp = api.financial_news(s = "AAPL.US", from_date = '2024-01-01', to_date = '2024-01-30', limit = 100)
df = pd.DataFrame(resp) # converting the json output into datframe
df.tail()

Let me explain the parameters in the API:


s: String. REQUIRED if parameter ‘t’ is not set. The ticker code to get news for.

t: String. REQUIRED if parameter ‘s’ is not set. The tag to get news on a given topic. you can find the provided topic list on this page: https://eodhd.com/financial-apis/stock-market-financial-news-api/

api_token: String. REQUIRED. Your api_token to access the API. You will get it after registration.

from and to: the format is ‘YYYY-MM-DD’. If you need data from Mar 1, 2021, to Mar 10, 2021, you should use from=2021–03–01 and to=2021–03–10.

limit: Number. OPTIONAL. The number of results should be returned with the query. Default value: 50, minimum value: 1, maximum value: 1000.

offset: Number. OPTIONAL. The offset of the data. Default value: 0, minimum value: 0. For example, to get 100 symbols starting from 200 you should use limit=100 and offset=200.

The output data has the following fields:


date: The date and time of the article are in ISO 8601 format.

title: The title of the article.

content: The full body of the article.

link: The link to the source.

symbols: The array of ticker symbols is mentioned in the article.

In [ ]:
# funtion to clean the textual data
def clean_text(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

# Apply the replacement function to the entire column
df['content'] = df['content'].apply(clean_text)

Now we will use Langchain to form an LLM chain with the OpenAI model.

In [ ]:

llm = ChatOpenAI(model = "gpt-3.5-turbo",
                 openai_api_key = 'YOUR OPENAI API KEY', 
                 temperature = 0)

leverage prompt engineering techniques, utilizing the Langchain template functionality, to construct an optimized prompt for conducting sentiment analysis in the stock market. The objective is to create a prompt that not only provides sentiment Analysis but also offers explanations for the model’s inferences.

In [ ]:
template = """
Identify the sentiment towards the Apple(AAPL) stocks from the news article , where the sentiment score should be from -10 to +10 where -10 being the most negative and +10 being the most positve , and 0 being neutral

Also give the proper explanation for your answers and how would it effect the prices of different stocks

Article : {statement}
"""

# forming prompt using Langchain PromptTemplate functionality
prompt = PromptTemplate(template = template, input_variables = ["statement"])
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [ ]:
Running the LLM chain :



print(llm_chain.run(df['content'][13]))

In [ ]:

# A function to count the number of tokens
def count_tokens(text):
    tokens = text.split()  
    return len(tokens)

Counting tokes for all the rows in a dataframe:


# Applying the tokenization function to the DataFrame columndf['TokenCount'] = df['content'].apply(count_tokens)

In [ ]:
# DataFrame from the TokenCount

# Define a token count threshold (for example, keep rows with more than 2 tokens)
token_count_threshold = 3500

# Create a new DataFrame by filtering based on the token count
new_df = df[df['TokenCount'] < token_count_threshold]

# Drop the 'TokenCount' column from the new DataFrame if you don't need it
new_df = new_df.drop('TokenCount', axis = 1)

# Resetting the index
new_df = new_df.reset_index(drop = True)

In [ ]:
# Change of Prompt Template
template_2 = """
Identify the sentiment towards the Apple(AAPL) stocks of the news article from -10 to +10 where -10 being the most negative and +10 being the most positve , and 0 being neutral

GIVE ANSWER IN ONLY ONE WORD AND THAT SHOULD BE THE SCORE

Article : {statement}
"""

# forming prompt using Langchain PromptTemplate functionality
prompt_2 = PromptTemplate(template = template_2, input_variables = ["statement"])

In [ ]:
# Let’s form the new LLM chain:



llm_chain_2 = LLMChain(prompt = prompt_2, llm = llm)

# Let me demonstrate one inference here:



print(new_df['content'][2])
print('')
print('News sentiment: ', llm_chain_2.run(new_df['content'][2]))

In [ ]:
# We will get the sentiment of each value
x = []
for i in range(0,new_df.shape[0]):
    x.append(llm_chain_2.run(new_df['content'][i]))

In [ ]:
# Showing Market Sentiment
import matplotlib.pyplot as plt

dt = pd.DataFrame(x) # Converting into Dataframe
column_name = 0 # this is my column name you should change it according to your data
value_counts = dt[column_name].value_counts()

# Plotting the pie chart
plt.pie(value_counts, labels = value_counts.index, autopct = '%1.1f%%', startangle = 140)
plt.title(f'Pie Chart')
plt.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# Show the pie chart
plt.show()

In [ ]:
# Removing Neutral Values
value_to_remove = '0'
# Remove all rows where the specified value occurs in the column
dt_new = dt[dt[0] != value_to_remove]

In [ ]:
value_counts = dt_new[column_name].value_counts()

# Plotting the pie chart
plt.pie(value_counts, labels = value_counts.index, autopct = '%1.1f%%', startangle = 140)
plt.title(f'Pie Chart')
plt.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.

# Show the pie chart
plt.show()